# SQL

In [1]:
%load_ext dotenv
%dotenv

In [2]:
from langchain import OpenAI, SelfAskWithSearchChain, LLMMathChain
from sqlalchemy.engine import Engine, create_engine
from langchain.agents import initialize_agent, Tool
import os

In [1]:
import sys
 # setting path
sys.path.append('..')
sys.path.append('../modules')

from modules.sql_database import SQLDatabase
from modules.sql_chain import SQLDatabaseChain

ModuleNotFoundError: No module named 'modules'

In [6]:
# connect to remote postgres database
db = SQLDatabase.from_uri(os.getenv("DB_CONNECTION_STRING"))
print("Tables found:", db._all_tables)
db_chain = SQLDatabaseChain(llm=OpenAI(), database=db, verbose=True)

Tables found: ['users', 'templates', 'models', 'generations']


ValidationError: 1 validation error for SQLDatabaseChain
database
  instance of SQLDatabase expected (type=type_error.arbitrary_type; expected_arbitrary_type=SQLDatabase)

In [10]:
# Load the tool configs that are needed.
llm = OpenAI(temperature=0)
llm_math_chain = LLMMathChain(llm=llm, verbose=True)
tools = [
    Tool(
        name = "Database",
        func=db_chain.run,
        description="useful for when you need to answer questions about data in your database"
    ),
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="useful for when you need to answer questions about math"
    )
]

In [11]:
llm = OpenAI(temperature=0)
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)

In [12]:
agent.run("Are there any users that are stuck in training?")



> Entering new ZeroShotAgent chain...
Are there any users that are stuck in training?
Thought: I need to check the database to see if any users are stuck in training.
Action: Database
Action Input: SELECT * FROM users WHERE status = 'stuck in training'

> Entering new SQLDatabaseChain chain...
SELECT * FROM users WHERE status = 'stuck in training' 
SQLQuery: SELECT * FROM users WHERE status = 'stuck in training';

ProgrammingError: (psycopg2.errors.UndefinedColumn) column "status" does not exist
LINE 1:  SELECT * FROM users WHERE status = 'stuck in training';
                                   ^

[SQL:  SELECT * FROM users WHERE status = 'stuck in training';]
(Background on this error at: http://sqlalche.me/e/13/f405)